# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Jeff Pasque</p>
### <p style="text-align: right;"> &#9989; Github: jeffpasque</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

git clone `https://github.com/jeffpasque/CMSE202-s23-turnin/blob/main/HW_4_Regression_and_SVM_STUDENT.ipynb`

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [46]:
# imports for the day
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [48]:
df = pd.read_csv("aussie_weather_data.csv")
df

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39510,2017,6,20,Darwin,19.3,33.4,0.0,6.0,11.0,22.5,...,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,0,0
39511,2017,6,21,Darwin,21.2,32.6,0.0,7.6,8.6,0.0,...,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,0,0
39512,2017,6,22,Darwin,20.7,32.8,0.0,5.6,11.0,0.0,...,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,0,0
39513,2017,6,23,Darwin,19.5,31.8,0.0,6.2,10.6,337.5,...,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,0,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [49]:
#Write your code here
print(df['Location'].unique())

['NorfolkIsland' 'SydneyAirport' 'WaggaWagga' 'MelbourneAirport' 'Mildura'
 'Watsonia' 'Brisbane' 'Cairns' 'Townsville' 'MountGambier' 'Nuriootpa'
 'PerthAirport' 'Perth' 'AliceSprings' 'Darwin']


#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [50]:
# Choosing Darwin, because thats a cool name

mask = df['Location'] == 'Darwin'
newdf = df[mask]
newdf

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
36453,2008,7,1,Darwin,20.0,33.1,0.0,4.4,11.0,0.0,...,81.0,32.0,1016.0,1012.1,1.0,2.0,25.4,32.3,0,0
36454,2008,7,2,Darwin,19.4,32.4,0.0,6.0,10.4,22.5,...,81.0,17.0,1016.8,1012.4,1.0,1.0,24.3,31.9,0,0
36455,2008,7,3,Darwin,18.2,31.8,0.0,8.0,11.0,0.0,...,38.0,24.0,1017.2,1013.0,0.0,1.0,24.3,31.2,0,0
36456,2008,7,4,Darwin,17.3,30.7,0.0,7.0,10.4,0.0,...,55.0,16.0,1017.3,1013.6,2.0,6.0,21.3,29.8,0,0
36457,2008,7,5,Darwin,15.5,30.8,0.0,7.0,10.8,337.5,...,37.0,16.0,1016.3,1012.6,1.0,1.0,22.2,29.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39510,2017,6,20,Darwin,19.3,33.4,0.0,6.0,11.0,22.5,...,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,0,0
39511,2017,6,21,Darwin,21.2,32.6,0.0,7.6,8.6,0.0,...,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,0,0
39512,2017,6,22,Darwin,20.7,32.8,0.0,5.6,11.0,0.0,...,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,0,0
39513,2017,6,23,Darwin,19.5,31.8,0.0,6.2,10.6,337.5,...,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [54]:
#Write your code here
X = newdf.drop('RainTomorrow', axis=1)
X = X.drop('Location', axis=1)
y = newdf['RainTomorrow']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)

print(len(X_train))
print(len(X_test))

2143
919


## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

Our response in binary (0 or 1). This is more of a classification problem, since we are trying to determine whether or not it will rain (yes or no essentially). Therefore, we will use logistic regression.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [55]:
#Write your code here
model = LogisticRegression(max_iter=10000).fit(X_train, y_train)

print("Intercept:", model.intercept_)
print("Coefficients:", model.coef_)
print("Accuracy:", model.score(X, y))
print(model)

mylist = []
for element in model.coef_:
    for item in element:
        mylist.append(item)
        
sorted_list = np.sort(mylist)[::-1]
largest_five = sorted_list[:5]

# Print largest five coefficients
print(largest_five)

Intercept: [0.00130519]
Coefficients: [[ 6.16224147e-02  7.53984709e-03  8.34721311e-03 -3.28274683e-02
   1.40299129e-01  1.41541694e-02 -5.21829356e-02 -2.22491291e-01
  -3.43283429e-04  8.88436087e-02 -7.83531485e-05 -1.75590061e-03
  -1.85585757e-02 -4.43086497e-02  1.51475845e-02  1.04687139e-01
   4.77505276e-01 -6.23028919e-01 -1.39526907e-01  1.34827166e-01
   2.25020364e-01  6.98960424e-02  1.10749058e-01]]
Accuracy: 0.8713259307642064
LogisticRegression(max_iter=10000)
[0.47750528 0.22502036 0.14029913 0.13482717 0.11074906]


*Record which features are most statistically significant.*

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [56]:
#Write your code here
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[614  65]
 [ 69 171]]


0.8541893362350381

#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

In [58]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Precision:", precision)
print("Recall:", recall)

Precision: 0.7245762711864406
Recall: 0.7125


The recall is higher, which means this model has a tendency to predict true positives more accurately at the cost of misclassifying true negatives as false positives.

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [59]:
#Write your code for calculating the frequency of rain here.

sum(newdf['RainTomorrow'] == 1)/len(newdf['RainTomorrow'])

0.25767472240365774

From the above code, we can see (from our isolated model where location was set to Darwin) that it rains 25% of the time. This means, we can say "Hey, it's not going to rain today" and be correct 65% of the time.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [60]:
C_list = [0.001, 0.01, 0.1, 1, 5, 10, 100]
tuned_parameters = [{'C': C_list}]
clf = GridSearchCV(SVC(kernel='rbf'), tuned_parameters, cv=10, scoring='accuracy', n_jobs = -1)
clf.fit(X, y)
print("RBF Model Best Params:" ,clf.best_params_)

C_list = [0.001, 0.01, 0.1, 1, 5, 10, 100]
tuned_parameters = [{'C': C_list}]
clf = GridSearchCV(SVC(kernel='linear'), tuned_parameters, cv=10, scoring='accuracy', n_jobs = -1)
clf.fit(X, y)
print("Linear Model Best Params:", clf.best_params_)


RBF Model Best Params: {'C': 100}
Linear Model Best Params: {'C': 0.1}


The best parameters for the RBF Model: C = 100

The best parameters for the Linear Model: C = 0.1 

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [61]:
#Write your code here
linear = SVC(kernel='linear', C = 0.1)
linear.fit(X,y)
rbf = SVC(kernel='rbf', C = 100)
rbf.fit(X,y)

y_pred1 = linear.predict(X_test)
y_pred2 = rbf.predict(X_test)

cm = confusion_matrix(y_test, y_pred1)
print(cm)

cm2 = confusion_matrix(y_test, y_pred2)
print(cm2)

[[627  52]
 [ 74 166]]
[[639  40]
 [ 89 151]]


In [62]:
from sklearn.metrics import accuracy_score
precision = precision_score(y_test, y_pred1)
recall = recall_score(y_test, y_pred1)
print("LINEAR")
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy_score(y_test, y_pred1))

print("\n")
print("RBF")
precision = precision_score(y_test, y_pred2)
recall = recall_score(y_test, y_pred2)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy_score(y_test, y_pred2))

LINEAR
Precision: 0.7614678899082569
Recall: 0.6916666666666667
Accuracy: 0.8628944504896626


RBF
Precision: 0.7905759162303665
Recall: 0.6291666666666667
Accuracy: 0.8596300326441785


## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


The logistic regression model performed better than the base model. The Linear Kernel for SVM has the highest accuracy of 0.87. This seems so high compared to the base model, but that's the overall goal.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


#### Original Seed:

**Linear Model**
Precision: 0.7291666666666666
Recall: 0.7478632478632479
Accuracy: 0.8650707290533188

**Linear SVM**
Precision: 0.7706422018348624
Recall: 0.717948717948718
Accuracy: 0.8737758433079434

**RBF SVM**
Precision: 0.8066298342541437
Recall: 0.6239316239316239
Accuracy: 0.8661588683351469


#### Seed set to 25

**Linear Model**
Precision: 0.7291666666666666
Recall: 0.7245762711864406
Accuracy: 0.7125

**Linear SVM**
Precision: 0.7614678899082569
Recall: 0.6916666666666667
Accuracy: 0.8628944504896626

**RBF SVM**
Precision: 0.7905759162303665
Recall: 0.6291666666666667
Accuracy: 0.8596300326441785


**Note**: The process of rerunning this takes roughly 10 minutes on my laptop.

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

Well now it seems the state of the seed is correlated with how accurate these models are. On some seeds, the logistic regression model is best, but other times the Linear or RBF SVM will be more accurate.

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

The SVM models performed better. Their results from the seed changes were very minimal compared to the Logistic Regression model.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University